In [1]:
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType, create_openai_functions_agent, AgentExecutor
#from langchain_experimental.tools import PythonREPLTool
from langchain import hub
from constants import OPENAI_API_KEY